In [124]:
import gym
import numpy as np
import random
import tensorflow as tf
import matplotlib.pyplot as plt
import scipy.misc
import os
from IPython import display
import time
%matplotlib inline

In [125]:
import numpy as np
import random
import itertools
import scipy.misc
import matplotlib.pyplot as plt


class gameOb():
    def __init__(self,coordinates,size,intensity,channel,reward,name):
        self.x = coordinates[0]
        self.y = coordinates[1]
        self.size = size
        self.intensity = intensity
        self.channel = channel
        self.reward = reward
        self.name = name
        
class gameEnv():
    def __init__(self,partial,size):
        self.sizeX = size
        self.sizeY = size
        self.actions = 4
        self.objects = []
        self.partial = partial
        self.a = self.reset()
#         plt.imshow(self.a,interpolation="nearest")
        
        
    def reset(self):
        self.objects = []
        hero = gameOb(self.newPosition(),1,1,2,None,'hero')
        self.objects.append(hero)
        bug = gameOb(self.newPosition(),1,1,1,1,'goal')
        self.objects.append(bug)
        hole = gameOb(self.newPosition(),1,1,0,0,'fire')
        self.objects.append(hole)
        bug2 = gameOb(self.newPosition(),1,1,1,1,'goal')
        self.objects.append(bug2)
        hole2 = gameOb(self.newPosition(),1,1,0,0,'fire')
        self.objects.append(hole2)
        bug3 = gameOb(self.newPosition(),1,1,1,1,'goal')
        self.objects.append(bug3)
        bug4 = gameOb(self.newPosition(),1,1,1,1,'goal')
        self.objects.append(bug4)
        state = self.renderEnv()
        self.state = state
        return state

    def moveChar(self,direction):
        # 0 - up, 1 - down, 2 - left, 3 - right
        hero = self.objects[0]
        heroX = hero.x
        heroY = hero.y
        penalize = 0.00
        if direction == 0 and hero.y >= 1:
            hero.y -= 1
        if direction == 1 and hero.y <= self.sizeY-2:
            hero.y += 1
        if direction == 2 and hero.x >= 1:
            hero.x -= 1
        if direction == 3 and hero.x <= self.sizeX-2:
            hero.x += 1     
        if hero.x == heroX and hero.y == heroY:
            penalize = 0
        self.objects[0] = hero
        return penalize
    
    def newPosition(self):
        iterables = [ range(self.sizeX), range(self.sizeY)]
        points = []
        for t in itertools.product(*iterables):
            points.append(t)
        currentPositions = []
        for objectA in self.objects:
            if (objectA.x,objectA.y) not in currentPositions:
                currentPositions.append((objectA.x,objectA.y))
        for pos in currentPositions:
            points.remove(pos)
        location = np.random.choice(range(len(points)),replace=False)
        return points[location]

    def checkGoal(self):
        others = []
        for obj in self.objects:
            if obj.name == 'hero':
                hero = obj
            else:
                others.append(obj)
        ended = False
        for other in others:
            if hero.x == other.x and hero.y == other.y:
                self.objects.remove(other)
                if other.reward == 1:
                    self.objects.append(gameOb(self.newPosition(),1,1,1,1,'goal'))
                else: 
                    self.objects.append(gameOb(self.newPosition(),1,1,0,0,'fire'))
                return other.reward,True
        if ended == False:
            return 0.0,False

    def renderEnv(self):
        #a = np.zeros([self.sizeY,self.sizeX,3])
        self.a = np.ones([self.sizeY+2,self.sizeX+2,3])
        self.a[1:-1,1:-1,:] = 0
        hero = None
        for item in self.objects:
            self.a[item.y+1:item.y+item.size+1,item.x+1:item.x+item.size+1,item.channel] = item.intensity
            if item.name == 'hero':
                hero = item
        if self.partial == True:
            self.a = self.a[hero.y:hero.y+3,hero.x:hero.x+3,:]
        b = scipy.misc.imresize(self.a[:,:,0],[84,84,1],interp='nearest')
        c = scipy.misc.imresize(self.a[:,:,1],[84,84,1],interp='nearest')
        d = scipy.misc.imresize(self.a[:,:,2],[84,84,1],interp='nearest')
        self.a = np.stack([b,c,d],axis=2)
        return self.a

    def step(self,action):
        penalty = self.moveChar(action)
        reward,done = self.checkGoal()
        state = self.renderEnv()
        return state,(reward-penalty),done

In [126]:
env = gameEnv(partial=False,size=5)

In [127]:
class Qnetwork():
    def __init__(self,h_size):
        #The network recieves a frame from the game, flattened into an array.
        #It then resizes it and processes it through four convolutional layers.
        self.scalarInput =  tf.placeholder(shape=[None,21168],dtype=tf.float32)
        self.imageIn = tf.reshape(self.scalarInput,shape=[-1,84,84,3])
        self.conv1 = tf.contrib.layers.convolution2d( \
            inputs=self.imageIn,num_outputs=32,kernel_size=[8,8],stride=[4,4],padding='VALID', biases_initializer=None)
        self.conv2 = tf.contrib.layers.convolution2d( \
            inputs=self.conv1,num_outputs=64,kernel_size=[4,4],stride=[2,2],padding='VALID', biases_initializer=None)
        self.conv3 = tf.contrib.layers.convolution2d( \
            inputs=self.conv2,num_outputs=64,kernel_size=[3,3],stride=[1,1],padding='VALID', biases_initializer=None)
        self.conv4 = tf.contrib.layers.convolution2d( \
            inputs=self.conv3,num_outputs=512,kernel_size=[7,7],stride=[1,1],padding='VALID', biases_initializer=None)
        
        #We take the output from the final convolutional layer and split it into separate advantage and value streams.
        self.streamAC,self.streamVC = tf.split(3,2,self.conv4)
        self.streamA = tf.contrib.layers.flatten(self.streamAC)
        self.streamV = tf.contrib.layers.flatten(self.streamVC)
        self.AW = tf.Variable(tf.random_normal([h_size//2,env.actions]))
        self.VW = tf.Variable(tf.random_normal([h_size//2,1]))
        self.Advantage = tf.matmul(self.streamA,self.AW)
        self.Value = tf.matmul(self.streamV,self.VW)
        
        #Then combine them together to get our final Q-values.
        self.Qout = self.Value + tf.sub(self.Advantage,tf.reduce_mean(self.Advantage,reduction_indices=1,keep_dims=True))
        self.predict = tf.argmax(self.Qout,1)
        
        #Below we obtain the loss by taking the sum of squares difference between the target and prediction Q values.
        self.targetQ = tf.placeholder(shape=[None],dtype=tf.float32)
        self.actions = tf.placeholder(shape=[None],dtype=tf.int32)
        self.actions_onehot = tf.one_hot(self.actions,env.actions,dtype=tf.float32)
        
        self.Q = tf.reduce_sum(tf.mul(self.Qout, self.actions_onehot), reduction_indices=1)
        
        self.td_error = tf.square(self.targetQ - self.Q)
        self.loss = tf.reduce_mean(self.td_error)
        self.trainer = tf.train.AdamOptimizer(learning_rate=0.0001)
        self.updateModel = self.trainer.minimize(self.loss)

In [128]:
class experience_buffer():
    def __init__(self, buffer_size = 50000):
        self.buffer = []
        self.buffer_size = buffer_size
    
    def add(self,experience):
        if len(self.buffer) + len(experience) >= self.buffer_size:
            self.buffer[0:(len(experience)+len(self.buffer))-self.buffer_size] = []
        self.buffer.extend(experience)
            
    def sample(self,size):
        return np.reshape(np.array(random.sample(self.buffer,size)),[size,5])

In [129]:
def processState(states):
    return np.reshape(states,[21168])

In [130]:
def updateTargetGraph(tfVars,tau):
    total_vars = len(tfVars)
    print(type(total_vars))
    op_holder = []
    print("tfVars length: ", total_vars)
    for idx,var in enumerate(tfVars[0:total_vars//2]):
        op_holder.append(tfVars[idx+total_vars//2].assign((var.value()*tau) + ((1-tau)*tfVars[idx+total_vars//2].value())))
    return op_holder

def updateTarget(op_holder,sess):
    for op in op_holder:
        sess.run(op)

In [131]:
batch_size = 32 #How many experiences to use for each training step.
update_freq = 4 #How often to perform a training step.
y = .99 #Discount factor on the target Q-values
startE = 1 #Starting chance of random action
endE = 0.1 #Final chance of random action
anneling_steps = 50000 #How many steps of training to reduce startE to endE.
num_episodes = 30003 #How many episodes of game environment to train network with.
pre_train_steps = 10000 #How many steps of random actions before training begins.
pre_train_steps_from_Q = False #If true, initialize buffer with steps from Q instead of random actions
max_epLength = 100 #The max allowed length of our episode.
load_model = False #Whether to load a saved model.
path = "./dqn/awjuliani/one_reward_extended" #The path to save our model to.
h_size = 512 #The size of the final convolutional layer before splitting it into Advantage and Value streams.
tau = 0.001 #Rate to update target network toward primary network

In [123]:
tf.reset_default_graph()
mainQN = Qnetwork(h_size)
targetQN = Qnetwork(h_size)

init = tf.initialize_all_variables()

saver = tf.train.Saver()

trainables = tf.trainable_variables()

targetOps = updateTargetGraph(trainables,tau)

myBuffer = experience_buffer()

#Set the rate of random action decrease. 
e = startE
stepDrop = (startE - endE)/anneling_steps

#create lists to contain total rewards and steps per episode
jList = []
rList = []
total_steps = 0

#Make a path for our model to be saved in.
if not os.path.exists(path):
    os.makedirs(path)

with tf.Session() as sess:
    if load_model == True:
        print('Loading Model...')
        load = './dqn/awjuliani/one_reward/model-10000.cptk'
        ckpt = tf.train.get_checkpoint_state(path)
        saver.restore(sess,load)
    sess.run(init)
    updateTarget(targetOps,sess) #Set the target network to be equal to the primary network.
    for i in range(1, num_episodes):
        episodeBuffer = experience_buffer()
        #Reset environment and get first new observation
        s = env.reset()
        s = processState(s)
        d = False
        rAll = 0
        j = 0
        #The Q-Network
        while j < max_epLength: #If the agent takes longer than 200 moves to reach either of the blocks, end the trial.
            j+=1
            #Choose an action by greedily (with e chance of random action) from the Q-network
            if pre_train_steps_from_Q:
                action = sess.run(mainQN.predict, feed_dict={mainQN.scalarInput:np.reshape(processState(env.a), [1, 21168])})
            elif np.random.rand(1) < e or total_steps < pre_train_steps:
                a = np.random.randint(0,4)
            else:
                a = sess.run(mainQN.predict,feed_dict={mainQN.scalarInput:[s]})[0]
            s1,r,d = env.step(a)
            s1 = processState(s1)
            total_steps += 1
            episodeBuffer.add(np.reshape(np.array([s,a,r,s1,d]),[1,5])) #Save the experience to our episode buffer.
            
            if total_steps > pre_train_steps:
                if e > endE:
                    e -= stepDrop
                
                if total_steps % (update_freq) == 0:
                    trainBatch = myBuffer.sample(batch_size) #Get a random batch of experiences.
                    #Below we perform the Double-DQN update to the target Q-values
                    Q1 = sess.run(mainQN.predict,feed_dict={mainQN.scalarInput:np.vstack(trainBatch[:,3])})
                    Q2 = sess.run(targetQN.Qout,feed_dict={targetQN.scalarInput:np.vstack(trainBatch[:,3])})
                    end_multiplier = -(trainBatch[:,4] - 1)
                    doubleQ = Q2[range(batch_size),Q1]
                    targetQ = trainBatch[:,2] + (y*doubleQ * end_multiplier)
                    #Update the network with our target values.
                    _ = sess.run(mainQN.updateModel, \
                        feed_dict={mainQN.scalarInput:np.vstack(trainBatch[:,0]),mainQN.targetQ:targetQ, mainQN.actions:trainBatch[:,1]})
                    
                    updateTarget(targetOps,sess) #Set the target network to be equal to the primary network.
            rAll += r
            s = s1
            
            if d == True:

                break
        
        #Get all experiences from this episode and discount their rewards.
        myBuffer.add(episodeBuffer.buffer)
        jList.append(j)
        rList.append(rAll)
        #Periodically save the model. 
        if i % 10000 == 0:
            saver.save(sess,path+'/model-'+str(i)+'.cptk')
            print("Saved Model")
        if i % 1000 == 0:
            print()
            print("last 1000 -- avg steps ",np.mean(jList[-1000:]), " -- %complete ", sum(rList[-1000:])/1000)
            print()
        if len(rList) % 100 == 0:
            print(i, total_steps,np.mean(rList[-100:]),np.mean(jList[-100:]))
    saver.save(sess,path+'/model-'+str(i)+'.cptk')
print("Percent of succesful episodes: " + str(sum(rList)/num_episodes) + "%")

<class 'int'>
tfVars length:  12
100 768 0.64 7.68
200 1584 0.64 8.16
300 2358 0.65 7.74
400 3168 0.61 8.1
500 3899 0.72 7.31
600 4698 0.72 7.99
700 5338 0.72 6.4
800 6028 0.67 6.9
900 6761 0.69 7.33

last 1000 -- avg steps  7.561  -- %complete  0.67

1000 7561 0.64 8.0
1100 8263 0.69 7.02
1200 9075 0.7 8.12
1300 10237 0.62 11.62
1400 12276 0.5 20.39
1500 14105 0.53 18.29
1600 15486 0.59 13.81
1700 16852 0.48 13.66
1800 18149 0.63 12.97
1900 19273 0.57 11.24

last 1000 -- avg steps  12.828  -- %complete  0.591

2000 20389 0.6 11.16
2100 21501 0.53 11.12
2200 22741 0.5 12.4
2300 24022 0.64 12.81
2400 24966 0.58 9.44
2500 26301 0.56 13.35
2600 27538 0.65 12.37
2700 28818 0.62 12.8
2800 30072 0.61 12.54
2900 31351 0.62 12.79

last 1000 -- avg steps  12.303  -- %complete  0.596

3000 32692 0.65 13.41
3100 34405 0.59 17.13
3200 35748 0.68 13.43
3300 37530 0.53 17.82
3400 39196 0.7 16.66
3500 41101 0.58 19.05
3600 43439 0.51 23.38
3700 45682 0.62 22.43
3800 47797 0.62 21.15
3900 50151 0.57 2

In [133]:
directions = ['up', 'down', 'left', 'right']
with tf.Session() as sess:
    load = './dqn/awjuliani/one_reward/model-10000.cptk'
    saver.restore(sess, load)
    sess.run(init)
    for i in range(20):
        complete = False
        iter = 0
        env = gameEnv(partial=False,size=5)
#         plt.imshow(env.a)
        time.sleep(2)
        r_all = 0
        while(complete == False):
            action = sess.run(mainQN.predict, feed_dict={mainQN.scalarInput:np.reshape(processState(env.a), [1, 21168])})
            observation, reward, done = env.step(action)
#             plt.imshow(env.a)
#             display.clear_output(wait=True)
#             display.display(plt.gcf())
            r_all += reward
            if(iter > 20):
                done = True
            iter += 1
            if(done):
                complete = True
                print(iter)
                print("Complete")
                print("reward: ", r_all)
#             time.sleep(.5)
#             plt.show()

5
Complete
reward:  1.0
2
Complete
reward:  1.0
22
Complete
reward:  0.0
22
Complete
reward:  0.0
22
Complete
reward:  0.0
1
Complete
reward:  0.0
3
Complete
reward:  0.0
1
Complete
reward:  1.0
22
Complete
reward:  0.0
3
Complete
reward:  1.0
22
Complete
reward:  0.0
22
Complete
reward:  0.0
22
Complete
reward:  0.0
22
Complete
reward:  0.0
22
Complete
reward:  0.0
22
Complete
reward:  0.0
22
Complete
reward:  0.0
1
Complete
reward:  1.0
5
Complete
reward:  1.0
1
Complete
reward:  0.0


In [118]:
ckpt.model_checkpoint_path

'./dqn/awjuliani/one_reward/model-0.cptk'